In [ ]:
!npx playwright install-deps

In [ ]:
# Check crawl4ai version
import crawl4ai
print(crawl4ai.__version__.__version__)

In [ ]:
!crawl4ai-setup

In [ ]:
!crawl4ai-doctor

In [4]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio
from urllib.parse import urlparse, urldefrag
from typing import List, Dict, Any
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, MemoryAdaptiveDispatcher, BrowserConfig, CacheMode
from crawl4ai.deep_crawling import BFSDeepCrawlStrategy
from crawl4ai.content_scraping_strategy import LXMLWebScrapingStrategy
from crawl4ai.deep_crawling.filters import (FilterChain, DomainFilter, URLPatternFilter)
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import re

async def crawl_recursive_internal_links(start_urls, max_depth=3, max_concurrent=120) -> List[Dict[str,Any]]:
    browser_config = BrowserConfig(
        browser_type="firefox",
        headless=True,
        extra_args=[
            "--no-sandbox",
            "--disable-setuid-sandbox",
            "--disable-gpu",
            "--disable-dev-shm-usage",
            "--disable-software-rasterizer",
            "--disable-extensions",
            "--disable-background-networking",
            "--disable-background-timer-throttling",
            "--disable-sync",
            "--single-process",
            "--no-zygote"
        ],
    )
    run_config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS, 
        stream=False,
        markdown_generator = DefaultMarkdownGenerator(
            options={"ignore_links": True,
                    "skip_internal_links": True,
                    "ignore_images": True}
        )
    )
    dispatcher = MemoryAdaptiveDispatcher(
        memory_threshold_percent=80.0,
        check_interval=1.0,
        max_session_permit=max_concurrent
    )

    visited = set()

    def normalize_url(url):
        return urldefrag(url)[0]

    current_urls = set([normalize_url(u) for u in start_urls])
    results_all = []

    async with AsyncWebCrawler(config=browser_config) as crawler:
        for depth in range(max_depth):
            urls_to_crawl = [normalize_url(url) for url in current_urls if normalize_url(url) not in visited]
            if not urls_to_crawl:
                break

            results = await crawler.arun_many(urls=urls_to_crawl, config=run_config, dispatcher=dispatcher)
            next_level_urls = set()

            for result in results:
                norm_url = normalize_url(result.url)
                visited.add(norm_url)

                if result.success and result.markdown:
                    results_all.append({'markdown': result.markdown, 'metadata': result.metadata})
                    for link in result.links.get("internal", []):
                        next_url = normalize_url(link["href"])
                        if re.search("\/node\/",next_url) == None and re.search("\/phonghoc\/",next_url) == None and next_url not in visited:
                            next_level_urls.add(next_url)

            current_urls = next_level_urls

    return results_all

results = asyncio.run(crawl_recursive_internal_links(["https://daa.uit.edu.vn/"]))

[INIT].... → Crawl4AI 0.7.4 

[FETCH]... ↓ https://daa.uit.edu.vn/                                                                              |
✓ | ⏱: 4.22s 

[SCRAPE].. ◆ https://daa.uit.edu.vn/                                                                              |
✓ | ⏱: 0.04s 

[COMPLETE] ● https://daa.uit.edu.vn/                                                                              |
✓ | ⏱: 4.26s 

[ERROR]... × https://daa.uit.edu.vn/...hcntt_19_2_2019scan.pdf  | Error: Unexpected error in _crawl_web at line 696
in _crawl_web (.local/lib/python3.10/site-packages/crawl4ai/async_crawler_strategy.py):
Error: Failed on navigating ACS-GOTO:
Page.goto: Download is starting
Call log:
  - navigating to "https://daa.uit.edu.vn/sites/daa/files/uploads/53_qd_dhcntt_19_2_2019scan.pdf", waiting until 
"domcontentloaded"


Code context:
 691                               tag="GOTO",
 692                               params={"url": url},
 693                           )
 694                           response = None
 695                       else:
 696 →                         raise RuntimeError(f"Failed on navigating ACS-GOTO:\n{str(e)}")
 697   
 698                   await self.execute_hook(
 699                       "after_goto", page, context=context, url=url, response=response, config=config
 700                   )
 701    

...

[SCRAPE].. ◆ https://daa.uit.edu.vn/thoikhoabieu                                                                  |
✓ | ⏱: 0.10s 

[COMPLETE] ● https://daa.uit.edu.vn/thoikhoabieu                                                                  |
✓ | ⏱: 52.88s 

[FETCH]... ↓ https://daa.uit.edu.vn/chinhquy                                                                      |
✓ | ⏱: 52.97s 

[SCRAPE].. ◆ https://daa.uit.edu.vn/chinhquy                                                                      |
✓ | ⏱: 0.05s 

[COMPLETE] ● https://daa.uit.edu.vn/chinhquy                                                                      |
✓ | ⏱: 53.02s 

In [11]:
for r in results:
    r['markdown'] = r['markdown'].replace("Skip to content Skip to navigation\nCổng thông tin đào tạo\nNavigation menu\n  * Home\n  * Giới thiệu \n    * Cổng thông tin đào tạo\n    * Các ngành đào tạo\n    * Phòng đào tạo đại học\n  * Thông báo \n    * Đại học chính quy\n    * Văn bằng 2\n    * Đào tạo từ xa\n  * Quy định - Hướng dẫn \n    * Quy chế, Quy định đào tạo đại học của Trường ĐHCNTT\n    * Quy chế, Quy định đào tạo đại học của ĐHQG-HCM\n    * Quy chế, Quy định đào tạo đại học của Bộ GDĐT\n    * Qui chế về công tác giáo trình\n    * Quy định đào tạo ngắn hạn\n    * Quy trình cho giảng viên\n    * Quy trình cho sinh viên\n    * Tra cứu và xác minh VB tốt nghiệp ĐH\n    * Hướng dẫn sinh viên về các quy định ngoại ngữ\n    * Hướng dẫn triển khai dạy và học qua mạng\n  * Kế hoạch năm\n  * Chương trình đào tạo \n    * Hệ chính quy\n      * CTĐT Khoá 2025\n      * CTĐT Khoá 2024\n      * CTĐT Khoá 2023\n      * CTĐT Khoá 2022\n      * CTĐT Khoá 2021\n      * CTĐT Khoá 2020\n      * CTĐT Khoá 2019\n      * CTĐT Khoá 2018\n      * CTĐT Khoá 2017\n      * CTĐT Khoá 2016\n      * CTĐT Khoá 2015\n      * CTĐT Khoá 2014\n      * CTĐT Khoá 2013\n      * CTĐT Khoá 2012\n      * CTĐT Khoá 2011 trở về trước\n      * Danh mục môn học\n      * Tóm tắt môn học\n      * Đề án mở ngành\n    * Hệ từ xa\n      * CTĐT Khoá 2008\n      * CTĐT Khoá 2013\n      * CTĐT Khoá 2018\n      * CTĐT Khoá 2019\n      * CTĐT Khoá 2020\n      * CTĐT Khoá 2021\n      * CTĐT Khoá 2022\n      * CTĐT Khoá 2023\n      * CTĐT Khoá 2024\n  * Lịch \n    * TKB\n    * Lịch phòng\n\n\n## Tìm kiếm\nTìm kiếm \n## Đăng Nhập\nTên truy cập *\nDùng tài khoản chứng thực\nMật khẩu *\nMật khẩu chứng thực\n## Liên kết\n--------- Liên kết website ------- -- Website trường -- Webmail -- Website môn học -- Tài khoản chứng thực -- Diễn đàn sinh viên -- Microsoft Azure -- Khoa Công Nghệ Phần Mềm -- Khoa Hệ Thống Thông Tin -- Khoa Kỹ Thuật Máy Tính -- Khoa Mạng Máy Tính & TT -- Khoa Khoa Học Máy Tính\n","")

In [13]:
for r in results:
    r['markdown'] = r['markdown'].replace("\n## Bài viết liên quan\n  * Thông báo lịch học HT2 Quản lý dự án Phát triển Phần mềm (SE358.Q12.1) ngày 20/10/2025 (29-09-2025)\n  * Thông báo lịch học HT2 Quản lý dự án Phát triển Phần mềm (SE358.Q12.1) ngày 13/10/2025 (29-09-2025)\n  * Thông báo lịch học HT2 Điện toán đám mây và phát triển ứng dụng hướng dịch vụ (SE360.Q11.1) ngày 17/10/2025 (29-09-2025)\n  * Thông báo lịch học HT2 Điện toán đám mây và phát triển ứng dụng hướng dịch vụ (SE360.Q11.1) ngày 10/10/2025 (29-09-2025)\n  * Thông báo lịch học HT2 Quản lý dự án Phát triển Phần mềm (SE358.Q12.1) ngày 06/10/2025 (29-09-2025)\n\n\n## Trang\n  * 1\n  * 2\n  * 3\n  * 4\n  * 5\n  * 6\n  * 7\n  * 8\n  * 9\n  * …\n  * sau ›\n  * cuối »\n\n\n**PHÒNG ĐÀO TẠO ĐẠI HỌC**  \nPhòng A120, Trường Đại học Công nghệ Thông tin.  \nKhu phố 34, Phường Linh Xuân, Thành phố Hồ Chí Minh.  \nĐiện thoại: **(028) 372 51993, Ext: 113**(Hệ từ xa qua mạng), **112**(Hệ chính quy).  \nEmail: **phongdaotaodh@uit.edu.vn**\nBack to top\n","")

In [15]:
print(results[:5])

[{'markdown': 'Skip to content Skip to navigation\nCổng thông tin đào tạo\nNavigation menu\n  * Home\n  * Giới thiệu \n    * Cổng thông tin đào tạo\n    * Các ngành đào tạo\n    * Phòng đào tạo đại học\n  * Thông báo \n    * Đại học chính quy\n    * Văn bằng 2\n    * Đào tạo từ xa\n  * Quy định - Hướng dẫn \n    * Quy chế, Quy định đào tạo đại học của Trường ĐHCNTT\n    * Quy chế, Quy định đào tạo đại học của ĐHQG-HCM\n    * Quy chế, Quy định đào tạo đại học của Bộ GDĐT\n    * Qui chế về công tác giáo trình\n    * Quy định đào tạo ngắn hạn\n    * Quy trình cho giảng viên\n    * Quy trình cho sinh viên\n    * Tra cứu và xác minh VB tốt nghiệp ĐH\n    * Hướng dẫn sinh viên về các quy định ngoại ngữ\n    * Hướng dẫn triển khai dạy và học qua mạng\n  * Kế hoạch năm\n  * Chương trình đào tạo \n    * Hệ chính quy\n      * CTĐT Khoá 2025\n      * CTĐT Khoá 2024\n      * CTĐT Khoá 2023\n      * CTĐT Khoá 2022\n      * CTĐT Khoá 2021\n      * CTĐT Khoá 2020\n      * CTĐT Khoá 2019\n      * CTĐT

In [83]:
from pinecone import Pinecone, ServerlessSpec
from google import genai
import os
from dotenv import load_dotenv

load_dotenv()

True

In [84]:
# Init Client
pc = Pinecone(api_key = os.environ["PINECONE_API_KEY"])
index_name = "uit-knowledgebase"
index = pc.Index(index_name)
client = genai.Client(api_key=os.environ["GEMINI_API_KEY_PAID"])

In [20]:
import re
import sys

def smart_chunk_markdown(markdown: str, max_len: int = 1000) -> List[str]:
    """Hierarchically splits markdown by #, ##, ### headers, then by characters, to ensure all chunks < max_len."""
    def split_by_header(md, header_pattern):
        indices = [m.start() for m in re.finditer(header_pattern, md, re.MULTILINE)]
        indices.append(len(md))
        return [md[indices[i]:indices[i+1]].strip() for i in range(len(indices)-1) if md[indices[i]:indices[i+1]].strip()]

    chunks = []

    for h1 in split_by_header(markdown, r'^# .+$'):
        if len(h1) > max_len:
            for h2 in split_by_header(h1, r'^## .+$'):
                if len(h2) > max_len:
                    for h3 in split_by_header(h2, r'^### .+$'):
                        if len(h3) > max_len:
                            for i in range(0, len(h3), max_len):
                                chunks.append(h3[i:i+max_len].strip())
                        else:
                            chunks.append(h3)
                else:
                    chunks.append(h2)
        else:
            chunks.append(h1)

    final_chunks = []

    for c in chunks:
        if len(c) > max_len:
            final_chunks.extend([c[i:i+max_len].strip() for i in range(0, len(c), max_len)])
        else:
            final_chunks.append(c)

    return [c for c in final_chunks if c]

#Chunk and collect metadata

ids, documents, metadatas = [], [], []
chunk_idx = 0

for doc in results:
  chunks = smart_chunk_markdown(doc['markdown'], max_len=1000)
  for chunk in chunks:
    ids.append(f"daa-chunk-{chunk_idx}")
    documents.append(chunk)
    meta = {k: v for k, v in doc['metadata'].items() if v is not None}
    meta.update({'chunk_index': chunk_idx, 'markdown': chunk})
    metadatas.append(meta)
    chunk_idx += 1

In [30]:
indices_list = [{'s': 0, 'e': 100}, {'s': 100, 'e': 200}, {'s': 200, 'e': 300}, {'s': 300, 'e': 400}, {'s': 400, 'e': 500}, {'s': 500, 'e': 600}, {'s': 600, 'e': 700}, {'s': 700, 'e': 800}, {'s': 800, 'e': 900}, {'s': 900, 'e': 1000}, {'s': 1000, 'e': 1100}, {'s': 1100, 'e': 1200}, {'s': 1200, 'e': 1300}, {'s': 1300, 'e': 1400}, {'s': 1400, 'e': 1500}, {'s': 1500, 'e': 1600}, {'s': 1600, 'e': 1700}, {'s': 1700, 'e': 1800}, {'s': 1800, 'e': 1900}, {'s': 1900, 'e': 2000}, {'s': 2000, 'e': 2100}, {'s': 2100, 'e': 2200}, {'s': 2200, 'e': 2300}, {'s': 2300, 'e': 2400}, {'s': 2400, 'e': 2500}, {'s': 2500, 'e': 2600}, {'s': 2600, 'e': 2700}, {'s': 2700, 'e': 2800}, {'s': 2800, 'e': 2900}, {'s': 2900, 'e': 3000}, {'s': 3000, 'e': 3100}, {'s': 3100, 'e': 3200}, {'s': 3200, 'e': 3300}, {'s': 3300, 'e': 3400}, {'s': 3400, 'e': 3500}, {'s': 3500, 'e': 3600}, {'s': 3600, 'e': 3700}, {'s': 3700, 'e': 3800}, {'s': 3800, 'e': 3900}, {'s': 3900, 'e': 4000}, {'s': 4000, 'e': 4100}, {'s': 4100, 'e': 4200}, {'s': 4200, 'e': 4300}, {'s': 4300, 'e': 4400}, {'s': 4400, 'e': 4500}, {'s': 4500, 'e': 4600}, {'s': 4600, 'e': 4700}, {'s': 4700, 'e': 4800}, {'s': 4800, 'e': 4870}]
indices_idx = 0
indices_max = len(indices_list)

In [82]:
curr_idx = indices_list[indices_idx]
embeds = []

embedding_res = client.models.embed_content(
    model="gemini-embedding-001",
    contents=documents[curr_idx['s']:curr_idx['e']])

for rs in embedding_res.embeddings:
    embeds.append(rs.values)
if len(ids[curr_idx['s']:curr_idx['e']]) == len(embeds) and len(embeds) == len(metadatas[curr_idx['s']:curr_idx['e']]):
    index.upsert(vectors=zip(ids[curr_idx['s']:curr_idx['e']], embeds, metadatas[curr_idx['s']:curr_idx['e']]), namespace="daa")
    indices_idx += 1
    print(indices_idx if indices_idx < indices_max else "done")
else:
    print('incorrect lengths')

done
